In [1]:
%pip install transformers
%pip install tf-keras

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### import libraries

In [2]:
from google.cloud import bigquery
import re

### read data

In [3]:
def read_bq():
    client = bigquery.Client()
    
    query = f"""
        SELECT *
        FROM `intern-project-415606.Criminal_Dataset.criminal_data`
    """
    
    try:
        query_job = client.query(query)
        df = query_job.to_dataframe()
        return df
    except Exception as e:
        print(f"Error: {e}")

In [4]:
df = read_bq()
df

,JLR_LINK,TRANS_TYPE_OF_CASE,TRANS_LEGAL_RELATIONSHIP,PDF_TEXT,EXTRACT,ID,NAME,Year,Month,Day,GENDER,BIRTH
0,https://congbobanan.toaan.gov.vn/2ta827827t1cv...,採取行政處理措施的決定,放入強制性戒毒機構,<Page:1>TÒA ÁN NHÂN DÂN CỘNG HÒA XÃ HỘI CHỦ NG...,Họ và tên: Nguyễn Văn C – Giới tính: Nam Sinh...,No_Id,Nguyễn Văn C,1994,1,25,Male,1994-01-25 00:00:00
1,https://congbobanan.toaan.gov.vn/2ta889668t1cv...,刑事,犯罪非法持有毒品,<Page:1>TÒA ÁN NHÂN DÂN CỘNG HÒA XÃ HỘI CHỦ NG...,"Họ và tên: Trần Ngọc T, sinh ngày 08/12/1993....",No_Id,Trần Ngọc T,1993,12,8,Male,1993-12-08 00:00:00
2,https://congbobanan.toaan.gov.vn/2ta153804t1cv...,刑事,盜竊財產罪,<Page:1>TOÀ ÁN NHÂN DÂN CỘNG HOÀ XÃ HỘI CHỦ NG...,"Bùi Văn T, sinh ngày 09/12/1989 Tại: Xóm T – ...",No_Id,Bùi Văn T,1989,12,9,Male,1989-12-09 00:00:00
3,https://congbobanan.toaan.gov.vn/2ta1126341t1c...,刑事,犯罪非法持有毒品,<Page:1>TÒA ÁN NHÂN DÂN CỘNG HÒA XÃ HỘI CHỦ NG...,"Nguyễn Văn Đ, sinh ngày 04/8/1978 tại huyện H...",No_Id,Nguyễn Văn Đ,1978,8,4,Male,1978-08-04 00:00:00
4,https://congbobanan.toaan.gov.vn/2ta1066372t1c...,刑事,欺詐佔有財產罪,<Page:1>TÒA ÁN NHÂN DÂN CỘNG HÒA XÃ HỘI CHỦ NG...,"Cao Thị B, sinh ngày 01/01/1975, tại huyện C,...",No_Id,Cao Thị B,1975,1,1,Female,1975-01-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
36545,https://congbobanan.toaan.gov.vn/2ta465562t1cv...,刑事,犯罪非法持有毒品,<Page:1>TÒA ÁN NHÂN DÂN CỘNG HÒA XÃ HỘI CHỦ NG...,Họ và tên: Trần Thành M; Tên gọi khác: Kh«ng;...,No_Id,Trần Thành M,1977,4,16,Male,1977-04-16 00:00:00
36546,https://congbobanan.toaan.gov.vn/2ta938857t1cv...,刑事,犯罪非法持有毒品,<Page:1>TÒA ÁN NHÂN DÂN CỘNG HÒA XÃ HỘI CHỦ NG...,"1. Nguyễn Đình T, sinh năm 1997 tại xã TK, hu...",No_Id,Nguyễn Đình Q,2005,9,14,Male,2005-09-14 00:00:00
36547,https://congbobanan.toaan.gov.vn/2ta107179t1cv...,刑事,故意對他人造成傷害或損害健康的罪行,<Page:1>TÒA ÁN NHÂN DÂN CỘNG HOÀ XÃ HỘI CHỦ NG...,"1- Huỳnh Duy Kh ( tức Kh), sinh ngày 15/7/199...",No_Id,Huỳnh Duy Kh,1997,7,15,Male,1997-07-15 00:00:00
36548,https://congbobanan.toaan.gov.vn/2ta1064276t1c...,刑事,盜竊財產罪,<Page:1>TÒA ÁN QUÂN SỰ CỘNG HÒA XÃ HỘI CHỦ NGH...,"1. Hồ Văn C, sinh ngày 30/7/2003 tại tỉnh Quả...",No_Id,Hồ Văn X,2003,3,6,Male,2003-03-06 00:00:00


In [5]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("token-classification", model="NlpHUST/ner-vietnamese-electra-base")

2024-05-15 07:12:09.657543: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-15 07:12:10.031034: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-15 07:12:11.469618: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
def process_text_by_chunks(tokens, text):

    #check chunk size (64 per chunk)
    chunk_size = (len(tokens) // 64) + 1
    chunk_result = []
    char_index = 0
    for i in range(chunk_size):
        chunk_text = " ".join(tokens[i*64:min((i+1)*64, len(text))])
        try:
            chunk_ner = pipe(chunk_text)
        except:
            chunk_ner = []
        for ner in chunk_ner:
            ner['start'] += char_index 
        chunk_result.extend(chunk_ner)
        char_index += len(chunk_text)
    # for cr in chunk_result:
    #     print(cr)

    chunk_result_dict = {item['start']: item['entity'] for item in chunk_result}
    return chunk_result_dict

In [ ]:
def create_table(project_id, dataset_id, table_id_write):
    schema = [
        bigquery.SchemaField("extract_id", "INTEGER", mode="REQUIRED"),
        bigquery.SchemaField("text", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("ner_label", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("sequence", "INTEGER", mode="REQUIRED")
    ]
    table = bigquery.Table(f"{project_id}.{dataset_id}.{table_id_write}", schema=schema)
    try:
        table = bigquery_client.create_table(table)
        return True
    except Exception as e:
        print(f"Table {project_id}.{dataset_id}.{table_id_write} already exists.")
        return False

# return the last row of data
def check_table(project_id, dataset_id, table_id_write):
    # Query to fetch the maximum extract_id and count of rows in the table
    query = f"""
            SELECT 
                MAX(extract_id) AS max_extract_id, 
                COUNT(*) AS total_rows
            FROM {project_id}.{dataset_id}.{table_id_write};
        """
    
    # Execute the query
    query_job = bigquery_client.query(query)

    # Fetch the result
    result = query_job.result()
    
    # Extract values from the result
    for row in result:
        return row['max_extract_id'], row['total_rows']

def ner_bq_update(project_id, dataset_id, df, table_id_write, set_index=0, sequence = 0):
    for i in range(set_index, len(df)):
        text = df['PDF_TEXT'][i]
        # Split the text using regular expression to handle both space and newline characters
        tokens = re.split(r'\s+', text)
        # tokens = [re.sub(r'[<>]', 'a', token) for token in tokens]
        # Filter out empty tokens
        tokens = [token.strip() for token in tokens if token.strip()]
        
        chunk_result_dict = process_text_by_chunks(tokens, text)
        sequence, char_index = 1, 0
        rows = []
        for t in range(len(tokens)):
            if char_index not in chunk_result_dict.keys():
                row = {
                    "extract_id": i,  # You can use the index 'i' as the ID for each sentence
                    "text": tokens[t],
                    "ner_label": 'other',
                    "sequence": sequence
                }

            else:
                row = {
                    "extract_id": i,  # You can use the index 'i' as the ID for each sentence
                    "text": tokens[t],
                    "ner_label": chunk_result_dict[char_index],
                    "sequence": sequence
                }
            sequence += 1
            rows.append(row)
            char_index += len(tokens[t]) + 1

        errors = bigquery_client.insert_rows_json(f"{project_id}.{dataset_id}.{table_id_write}", rows)

        if i % 100 == 0:
            print(f"Inserted successfully for sentence {i}.")
        

if __name__ == "__main__":
    # Replace 'your-project-id' with your actual project ID
    PROJECT_ID = "intern-project-415606"
    # Dataset ID
    DATASET_ID = "Criminal_Dataset"
    # Table ID for reading the data
    TABLE_ID = "criminal_data"
    # Table ID for writing the data
    TABLE_ID_WRITE = "criminal_data_electra"

    bigquery_client = bigquery.Client(project=PROJECT_ID)
    
    # Read the data from the BigQuery table
    df = read_bq()
    
    if create_table(PROJECT_ID, DATASET_ID, TABLE_ID_WRITE):
        new_id, sequenc = 0, 1
    else:
        new_id, sequence = check_table(PROJECT_ID, DATASET_ID, TABLE_ID_WRITE)
        # new_id, sequence = 0, 1
        new_id, sequence = int(new_id), int(sequence)
    # Update the data with named entity recognition, set start index (default = 0)
    ner_bq_update(PROJECT_ID, DATASET_ID, df, TABLE_ID_WRITE, new_id, sequence)

Table intern-project-415606.Criminal_Dataset.criminal_data_electra already exists.


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Inserted successfully for sentence 1700.
Inserted successfully for sentence 1800.
Inserted successfully for sentence 1900.
Inserted successfully for sentence 2000.
Inserted successfully for sentence 2100.
Inserted successfully for sentence 2200.
Inserted successfully for sentence 2300.
Inserted successfully for sentence 2400.
